In [1]:
!pip install transformers diffusers torch torchvision accelerate

# I2VGen XL Model

More Info: https://huggingface.co/docs/diffusers/en/api/pipelines/i2vgenxl

# An Extra Note on Removing Models In case you run out of space

Here's a detailed step-by-step explanation on how to navigate to the `~/.cache` directory, find the `huggingface/hub` subdirectories, and delete a specific directory of cached models using the command line.

### Step-by-Step Instructions

1. **Open Terminal:**
   - On MacOS, you can open the Terminal application by searching for it in Spotlight (Cmd + Space and type "Terminal") or finding it in the Applications > Utilities folder.

2. **Navigate to the Home Directory:**
   - By default, the Terminal opens in your home directory. You can ensure you are in the home directory by typing:
     ```bash
     cd ~
     ```

3. **Navigate to the `.cache` Directory:**
   - The `.cache` directory is a hidden directory in your home folder. You can navigate to it using:
     ```bash
     cd .cache
     ```

4. **List Contents of the `.cache` Directory:**
   - To see the contents of the `.cache` directory, you can use the `ls` command:
     ```bash
     ls
     ```

5. **Navigate to the `huggingface/hub` Directory:**
   - Now, navigate to the `huggingface/hub` directory:
     ```bash
     cd huggingface/hub
     ```

6. **List Contents of the `hub` Directory:**
   - To see the contents of the `hub` directory, use the `ls` command again:
     ```bash
     ls
     ```

7. **Find the Specific Cached Model Directory:**
   - Look for the directory you want to delete, in this example `models--stabilityai--stable-video-diffusion-img2vid-xt`. You can verify its presence by listing the contents:
     ```bash
     ls
     ```

8. **Delete the Specific Cached Model Directory:**
   - Once you have confirmed the directory name, you can delete it using the `rm` command with the `-r` (recursive) flag. Be very careful with this command, as it will permanently delete the directory and its contents.
     ```bash
     rm -r models--stabilityai--stable-video-diffusion-img2vid-xt/
     ```

### Example Commands Summary:
```bash
cd ~
cd .cache
cd huggingface/hub
ls
rm -r models--stabilityai--stable-video-diffusion-img2vid-xt/
```

### Additional Tips:
- **Use Tab Completion:** While typing directory names, you can press the `Tab` key to autocomplete the names, which can help avoid typos.
- **Confirm Before Deleting:** If you're not sure about deleting a directory, you can list its contents with `ls` to confirm what it contains.
- **Backup Important Data:** Make sure you don't need any data from the directory you're deleting, as this operation is irreversible.

Following these steps should help you navigate to the `~/.cache` directory, locate the `huggingface/hub` subdirectories, and delete a specific cached model directory using the command line.

### Release CUDA Memory

In [2]:
!nvidia-smi

Sun Jul 14 08:25:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
# Empty GPU RAM before anything
torch.cuda.empty_cache()


In [4]:
import torch
from diffusers import I2VGenXLPipeline
from diffusers.utils import load_image, export_to_gif, export_to_video

In [5]:
# Load the model pipeline
# https://huggingface.co/docs/diffusers/en/api/pipelines/i2vgenxl
repo_id = "ali-vilab/i2vgen-xl"
pipeline = I2VGenXLPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, variant="fp16")
pipeline.enable_model_cpu_offload()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
image_url = "astro_moon.png"
image = load_image(image_url).convert("RGB")
prompt = "astronaut jumping straight up"

In [7]:
generator = torch.manual_seed(8888)
# This takes about 12 minutes in a T4
frames = pipeline(
    prompt=prompt,
    image=image,
    generator=generator
).frames[0] # The frames are in the first elemet of the pipeline.frames
# Convert list of PIL.Images to GIF
print(export_to_gif(frames))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


  0%|          | 0/50 [00:00<?, ?it/s]

/tmp/tmpa9o5cxx7.gif


In [8]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ab3a1c17-1451-a5cf-c95c-931470d8041f)


In [9]:
len(frames) # Only 16 frames

16

In [10]:
export_to_video(frames,'astro_jump.mp4',fps=8)

'astro_jump.mp4'